<h1>VSS SEB Parsing for Ephemerides</h1>

In [3]:
import pandas as pd
from astroquery.simbad import Simbad

In [4]:
df = pd.DataFrame()

In [5]:
df = pd.read_excel('VSS SEB observed targets.xlsx', sheet_name='Data')

In [6]:
df.head()

,Star,Epoch (HJD),Epoch Error,Period (d),Period Error,O-C Diagram Notes,Recommended Fequency,Last Observed
0,Ant S,2.458519e+06,0.000287,0.648348,2.959964e-07,linear?,yearly,2022-03-08 15:49:44.602
1,Aps DW,2.439210e+06,0.000787,2.312944,9.672835e-08,linear?,yearly,2022-05-09 14:58:03.446
2,Aps II,2.451931e+06,0.149860,0.842229,1.695586e-05,eccentric + apsidal motion?,2 per year,2021-06-23 10:19:15.616
3,Aps MR,2.457159e+06,0.000462,0.527864,1.404435e-07,linear?,yearly,2023-07-26 11:58:57.014
4,Aps NT,2.457208e+06,0.000361,0.294762,5.285139e-08,downward parabola?,yearly,2024-05-23 13:12:24.365


In [78]:
df.columns

Index([                'Star',          'Epoch (HJD)',          'Epoch Error',
                 'Period (d)',         'Period Error',    'O-C Diagram Notes',
       'Recommended Fequency',        'Last Observed',     ('Const', 'VarID')],
      dtype='object')

In [7]:
df['Const', 'VarID'] = df['Star'].str.split()

We parse the weirdness of the input file

In [87]:
objectsdf = pd.DataFrame()
for i in range(len(df)):
    details = df['Const', 'VarID'].iloc[i]
    if (len(details) == 2):
        object_name = details[1] +' '+ details[0]
    if (len(details) > 2):
        if (details[2].lower() == "secondary" or details[2].lower() == "primary"):
            object_name = details[1] +' '+ details[0]
        else:
            object_name = details[1]+details[2]# +' '+ details[0]

    result_table = Simbad.query_object(object_name)
    #print(result_table)
    if (result_table is None):
        print(f'{object_name}: failed to locate')
    else:
        objdf = result_table.to_pandas()
        for colname in df.columns[:-1]:
            #print(colname, df.iloc[i][colname])
            objdf[colname] = df.iloc[i][colname]
        objectsdf = pd.concat([objectsdf, objdf], ignore_index=True)
columns_to_drop = ['RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA','COO_ERR_ANGLE','COO_QUAL','COO_WAVELENGTH','COO_BIBCODE',	'SCRIPT_NUMBER_ID']
objectsdf = objectsdf.drop(columns=columns_to_drop)

NSV25657: failed to locate


ASASSN-VJ184530.00-593235.0: failed to locate


NSV1389: failed to locate


In [89]:
objectsdf['RA'] = objectsdf['RA'].str.replace(' ',':')
objectsdf['DEC'] = objectsdf['DEC'].str.replace(' ',':')
objectsdf['MAIN_ID'] = objectsdf['MAIN_ID'].str.replace('V* ','')

In [90]:
objectsdf

,MAIN_ID,RA,DEC,Star,Epoch (HJD),Epoch Error,Period (d),Period Error,O-C Diagram Notes,Recommended Fequency,Last Observed
0,S Ant,09:32:18.3864,-28:37:39.968,Ant S,2.458519e+06,0.000287,0.648348,2.959964e-07,linear?,yearly,2022-03-08 15:49:44.602
1,DW Aps,17:23:30.0309,-67:55:44.906,Aps DW,2.439210e+06,0.000787,2.312944,9.672835e-08,linear?,yearly,2022-05-09 14:58:03.446
2,II Aps,16:27:41.9350,-74:50:44.229,Aps II,2.451931e+06,0.149860,0.842229,1.695586e-05,eccentric + apsidal motion?,2 per year,2021-06-23 10:19:15.616
3,MR Aps,17:07:57.5281,-73:43:35.400,Aps MR,2.457159e+06,0.000462,0.527864,1.404435e-07,linear?,yearly,2023-07-26 11:58:57.014
4,HD 128575,14:42:42.5763,-74:18:41.568,Aps NT,2.457208e+06,0.000361,0.294762,5.285139e-08,downward parabola?,yearly,2024-05-23 13:12:24.365
...,...,...,...,...,...,...,...,...,...,...,...
206,V448 Vel,08:24:55.4977,-48:33:34.971,Vel V0448,2.451868e+06,NaN,0.364876,NaN,only two ToMs,yearly,2021-03-04 14:09:17.856
207,BD-07 3477,12:44:20.2386,-08:40:16.846,Vir HW,2.452500e+06,0.000190,0.116720,3.396294e-09,linear?,yearly,2023-06-24 11:18:51.638
208,AE Vol,08:45:39.4033,-64:59:28.043,Vol AE,2.452614e+06,0.000285,0.406615,1.940033e-08,linear?,yearly,2022-12-28 13:18:28.541
209,AF Vol,08:53:34.8637,-70:28:07.633,Vol AF,2.452618e+06,0.000565,0.638935,6.027154e-08,linear?,yearly,2023-01-18 12:52:15.456


<hr>

Okay now time to format the dataframe to the format of the empherides cat

Here is a sample:

 CONS NAME             PS RATING RA2000   DE2000  M0           PER          MAX      PRIM     SEK      BRANCH LD   SD   GSC         TYPE         N10    N     
 And  RT               p  1      231110.1 +530133 51463.24     0.62892951   8.55     9.47     8.88     V      2.6       3998.02167  EA/RS        136    1576  


and it is formated Fortran style - columns have fixed width


cons = 6; name=17; PS=3, rating=7, RA2000=9, DE2000=8
M0=13, per=13, max=9, prim=9, sek=9, branch=7, ld=5, sd=5, GSC=12, type=13, N10=7, n=7

In [91]:
# Define column formatters
formatters = {
    'CONS': '{:<6}'.format,  
    'NAME': '{:<17}'.format,  
    'PS': '{:<3}'.format,
    'RATING': '{:<7}'.format,  
    'RA2000': '{:<9}'.format,  
    'DE2000': '{:<8}'.format,
    'M0': '{:<13}'.format,  
    'PER': '{:<13}'.format,  
    'MAX': '{:<9}'.format,
    'PRIM': '{:<9}'.format,  
    'SEK': '{:<9}'.format,  
    'BRANCH': '{:<7}'.format,
    'LD': '{:<5}'.format,  
    'SD': '{:<5}'.format,
    'GSC': '{:<12}'.format,  
    'TYPE': '{:<13}'.format,
    'N10': '{:<7}'.format,  
    'N': '{:<7}'.format,
}

In [130]:
ephem = pd.DataFrame()
# ah we have to go element by element
const = []
name = []
ps =[]
rating = []
ra2000 = []
for i in range(0, len(objectsdf)):
    rating.append(10)
    ra2000.append(format(float(objectsdf.iloc[i]['RA'].replace(':','')), ".1f" ) )
    
    try: 
        (a,b) = objectsdf.iloc[i]['Star'].split(' ')
        const.append(a)
        name.append(b)
        ps.append('p')
    except:
        try:
            (a,b,c) = objectsdf.iloc[i]['Star'].split(' ')
            const.append(a)
            if (c.lower()=='primary' or c.lower()=='secondary'):
                name.append(b+'_'+c)
                if (c.lower()=='primary'): 
                    ps.append('p')
                if (c.lower()=='secondary'): 
                    ps.append('s')
            else:
                name.append(b+c)
                ps.append('p')
        except:
            print(f'cannot parse {objectsdf.iloc[i]['Star']}')
            


In [131]:
ephem['CONST'] = const
ephem['NAME'] = name
ephem['PS'] = ps
ephem['RATING'] = rating
ephem['RA2000'] = ra2000

In [132]:
# Convert DataFrame to a fixed-width formatted string
fixed_width_str = ephem.to_string(index=False, formatters=formatters)

In [133]:
fixed_width_str

'CONST               NAME  PS  RATING    RA2000\n  Ant  S                 p   10      93218.4  \n  Aps  DW                p   10      172330.0 \n  Aps  II                p   10      162741.9 \n  Aps  MR                p   10      170757.5 \n  Aps  NT                p   10      144242.6 \n  Aps  NX                p   10      145641.8 \n  Aps  TV                p   10      144549.7 \n  Aps  V0354             p   10      151528.6 \n  Aps  YY                p   10      150234.3 \n  Aqr  AO                p   10      221131.9 \n  Aqr  DX                p   10      220226.2 \n  Aqr  EE                p   10      223442.0 \n  Ara  FU                p   10      173332.3 \n  Ara  R                 p   10      163944.7 \n  Ara  V0489             p   10      171042.5 \n  Ara  V0491             p   10      171104.7 \n  Ara  V0535             p   10      173805.6 \n  Ara  V0610             p   10      165211.3 \n  Ara  V0870             p   10      180822.7 \n  Ara  V0878             p   10      16

In [134]:
# Save to a file
with open('fixed_width_file.txt', 'w') as file:
    file.write(fixed_width_str)
